## Spark Operation 
- Transformations = 새로운 RDD 반환 - 지연 실행
- Actions = 연산 결과 출력 및 저장 - 즉시 실행

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("transformations_actions")
sc = SparkContext(conf=conf)

# RDD 생성
일반 파이썬의 리스트를 이용해서 RDD 생성
- `parallelize([item1,item2])`

In [3]:
foods = sc.parallelize([
    "짜장면","짬뽕","마라탕",
    "떡볶이","쌀국수","짬뽕",
    "짜장면","짜장면","짬뽕",
    "마라탕","라면","라면",
    "우동","쌀국수"
    
])
foods.collect()

['짜장면',
 '짬뽕',
 '마라탕',
 '떡볶이',
 '쌀국수',
 '짬뽕',
 '짜장면',
 '짜장면',
 '짬뽕',
 '마라탕',
 '라면',
 '라면',
 '우동',
 '쌀국수']

In [4]:
foods.countByValue()

defaultdict(int,
            {'짜장면': 3,
             '짬뽕': 3,
             '마라탕': 2,
             '떡볶이': 1,
             '쌀국수': 2,
             '라면': 2,
             '우동': 1})

#  상위 N개 데이터 가져오기
- `take`

In [5]:
foods.take(3)

['짜장면', '짬뽕', '마라탕']

# 처음 1개 데이터 가져오기
- `first()`

In [6]:
foods.first()

'짜장면'

# 중복 데이터 제거하기
-`distinct()`
- **transformation인데 action 비슷함.**

In [9]:
items = foods.distinct() # RDD 생성은 곧 Transformation을 의미

In [10]:
items.collect()

['짜장면', '짬뽕', '마라탕', '떡볶이', '쌀국수', '라면', '우동']

# Narrow Transformations
- `map(<task>)`
- 데이터를 하나씩 꺼내서 `<task>`가 적용된 새로운 RDD 생성

In [11]:
sample_rdd= sc.parallelize([1,2,3,])
sample_rdd

ParallelCollectionRDD[14] at readRDDFromFile at PythonRDD.scala:274

In [12]:
sample_rdd.collect()

[1, 2, 3]

In [14]:
# task는 반드시 return이 있어야한다.
sample_rdd2=sample_rdd.map(lambda x : x**2)
sample_rdd2

PythonRDD[15] at RDD at PythonRDD.scala:53

In [15]:
sample_rdd2.collect()

[1, 4, 9]

# map 함수와 거의 비슷하나 flatMap 함수는 map의 모든 결과를 1차원 배열 형식으로 평평하게(flat)하게 나타낸다.
- `flatMap(<task>)`

In [16]:
movies = [
    "그린 북",
    "매트릭스",
    "토이 스토리",
    "캐스트 어웨이",
    "포드 V 페라리",
    "보헤미안 랩소디",
    "빽 투 더 퓨처",
    "반지의 제왕",
    "죽은 시인의 사회"
]

In [17]:
moviesRDD=sc.parallelize(movies)
moviesRDD

ParallelCollectionRDD[16] at readRDDFromFile at PythonRDD.scala:274

In [18]:
mapmovies=moviesRDD.map(lambda x: x.split())
mapmovies.collect()

[['그린', '북'],
 ['매트릭스'],
 ['토이', '스토리'],
 ['캐스트', '어웨이'],
 ['포드', 'V', '페라리'],
 ['보헤미안', '랩소디'],
 ['빽', '투', '더', '퓨처'],
 ['반지의', '제왕'],
 ['죽은', '시인의', '사회']]

In [23]:
# 모든 배열을 1차원으로 다 잘라서 구분 없이 리스트로 가져온다.
flatmovies=moviesRDD.flatMap(lambda x: x.split())
flatmovies.take(5)

['그린', '북', '매트릭스', '토이', '스토리']

# 조건에 맞는 결과만 얻어내기
- `filter(<task>)`

In [24]:
filteredmovies=flatmovies.filter(lambda x:x != '그린')
filteredmovies.take(5)

['북', '매트릭스', '토이', '스토리', '캐스트']

# Wide Trnasformations
- `groupBy(<task>)`
- 데이터를 내 기준에 맞게 묶어준다.
    - `task` : 데이터를 묶어주는 (Grouping) 기준을 설정. 

In [27]:
foodsgroup =foods.groupBy(lambda x:x[0])
result =foodsgroup.collect()
result

[('짜', <pyspark.resultiterable.ResultIterable at 0x2a1ed8f1580>),
 ('짬', <pyspark.resultiterable.ResultIterable at 0x2a1ed807460>),
 ('마', <pyspark.resultiterable.ResultIterable at 0x2a1ed807340>),
 ('떡', <pyspark.resultiterable.ResultIterable at 0x2a1ed807e20>),
 ('쌀', <pyspark.resultiterable.ResultIterable at 0x2a1ed807eb0>),
 ('라', <pyspark.resultiterable.ResultIterable at 0x2a1ed807f10>),
 ('우', <pyspark.resultiterable.ResultIterable at 0x2a1ec99a250>)]

In [31]:
for k,v in result:
    # print(k)
    # print(list(v))
    print(k,list(v))

짜 ['짜장면', '짜장면', '짜장면']
짬 ['짬뽕', '짬뽕', '짬뽕']
마 ['마라탕', '마라탕']
떡 ['떡볶이']
쌀 ['쌀국수', '쌀국수']
라 ['라면', '라면']
우 ['우동']


## transformation 과 action으로 나눈 이유? 탄력성!
- 메모리를 최대한 활용 가능
    - 연산을 지연 시킴으로 인해서 디스크와 네트워크의 연산을 최소화 할 수 있다.
- 반복적인 데이터 작업에 의한 비효율성
    - 저장하고 불러오는 방식은 속도를 저하

### storage Level
- MEMORY_ONLY
    - MEMORY(RAM)에만 데이터를 올려 놓기
- MEMORY_AND_DISK
    - MEMORY,DISK 동시에 데이터를 올려 놓기
    - 메모리에 용량이 부족하면 DISK에 데이터를 같이 올려 놓기 
- MEMORY_ONLY_SER, MEMORY_AND_DISK_SER
    - SER(Serialization)- 데이터의 용량을 아끼기 위해 직렬화 수행
    - 압축느낌으로 용량은 줄어들지만 데이터를 읽어올 떄 압축을 푸는(Deserialization)이 수행되어야 하기 때문에 불러오는 시간이 늘어난다.

## cache, persist
- 모두 메모리에 데이터를 올려두는 작업을한다.
- cache
    - `default Storage Level을 사용`
    - RDD : MEMORY_ONLY
    - DataFrame(DF) : MEMORY_AND_DISK
- persist
    - Storeage Level을 `사용자가 원하는 대로 지정이 가능`하다.